<span style="font-family:Lucida Bright;">

<hr style="border:2px solid black"> </hr>

<p style="margin-bottom:1cm"></p>

<center>
<font size="7"><b>Social Data Analysis and Visualization</b></font>
<p style="margin-bottom:1cm"></p>
<font size="6.8"><b>Final Project</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="5.8"><b>Data Import and Cleaning</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<br>
<font size="3"><b></b></font>

</center>

<p style="margin-bottom:0.7cm"></p>

<hr style="border:2px solid black"> </hr>

<hr style="border:2px solid black"> </hr>

# Initialization

## How to read this notebook

<span style="font-family:Arial;">

Please note that the pre-rendered outputs will first display properly when the notebook is __trusted__.

## Imports

In [1]:
%matplotlib inline

import bokeh.plotting as bplt
import calendar
import datetime
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import pickle
import plotly.express as px
import scipy.stats
import seaborn as sns
import urllib.request
import warnings

from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.models import Legend
from bokeh.models.ranges import FactorRange
from bokeh.models.sources import ColumnDataSource

from folium.map import FeatureGroup
from folium.plugins import HeatMap, HeatMapWithTime

from functools import reduce

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
from IPython.display import Markdown
from IPython.display import YouTubeVideo

from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.image import NonUniformImage

from mpl_toolkits.axes_grid1 import make_axes_locatable

from operator import itemgetter

from pathlib import Path

from scipy import stats

from sklearn.datasets import fetch_20newsgroups
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

##  Configuration

### Notebook options

In [ ]:
# Decide which output is shown below the cells.
InteractiveShell.ast_node_interactivity = "none"

### Bokeh options

In [ ]:
# Show bokeh figures in the notebook.
output_notebook()

### Matplotlib options

In [50]:
# Show matplotlib plots inline.
%matplotlib inline

### Pandas options

In [2]:
# Define the format in which the numbers will be shown in
# the pandas dataframes.
pd.options.display.float_format = '{:,.1f}'.format

# Decide how to handle the "SettingWithCopyWarning" warning
pd.options.mode.chained_assignment = None  # default='warn'

# Set the maximum number of rows and columns to show when 
# displaying a Pandas dataframe.
pd.options.display.max_rows = 75
pd.options.display.max_columns = 75

Loading BokehJS ...

### Warnings

In [ ]:
# Decide how to handle warnings.
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=PerformanceWarning)

## Paths

In [3]:
# Project root.
path_root = Path(
    r'C:\GDrive\DTU\Kurser\Social_Data_Analysis_and_Visualization_02806\final_project')

# Resources root.
path_resources_root = path_root / 'resources'

# Data folders.
path_data_root = path_resources_root / 'data'
path_data_citizenship_root = path_data_root / 'citizenship'
path_data_dwellings_root = path_data_root / 'dwellings'
path_data_geo_root = path_data_root / 'geo'
path_data_marital_status_root = path_data_root / 'marital_status'
path_data_without_district_info_root = path_data_root / 'without_district_information'
path_data_clean_root = path_data_root / 'clean'

## Function definitions

### General functions

In [4]:
# A function that returns a dict of object attributes.
def get_obj_attributes(obj):
    return {attribute_name: getattr(obj, attribute_name)
            for attribute_name in dir(obj)
            if (not attribute_name.startswith('__')
                and not callable(getattr(obj, attribute_name)))}

# A function that will print a markdown text.
def printmd(string):
    display(Markdown(string))
    
# A function that returns unique values from a text.
def unique(list_):
    return list(set(list_))

### Load data

In [51]:
# Define a class th
class CphData:

    def __init__(self):
        # Country of origin (no distric info)
        df_country_of_origin = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_country_of_origin_without_district.pkl')
        df_country_of_origin.attrs = {'name': 'Country of origin'}
        self.country_of_origin = df_country_of_origin

        # Citizenship
        df_citizenship = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_citizenship.pkl')
        df_citizenship.attrs = {'name': 'Citizenship'}
        self.citizenship = df_citizenship

        # Marital status
        df_marital_status = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_marital_status.pkl')
        df_marital_status.attrs = {'name': 'Marital status'}
        self.marital_status = df_marital_status

        # Family type and children
        df_family_type_and_children = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_family_type_and_number_of_chidren.pkl')
        df_family_type_and_children.attrs = {
            'name': 'Family type and children'}
        self.family_type_and_children = df_family_type_and_children

        # Income
        df_income = pd.read_pickle(path_data_clean_root / 'cph_income.pkl')
        df_income.attrs = {'name': 'Income'}
        self.income = df_income

        # Life span
        df_life_span = pd.read_pickle(
            path_data_clean_root / 'cph_life_span.pkl')
        df_life_span.attrs = {'name': 'Life span'}
        self.life_span = df_life_span

        # Population movement
        df_population_movement = pd.read_pickle(
            path_data_clean_root / 'cph_population_movement.pkl')
        df_population_movement.attrs = {'name': 'Population movement'}
        self.population_movement = df_population_movement

        # Dwellings
        df_dwellings = pd.read_pickle(
            path_data_clean_root / 'cph_dwellings.pkl')
        df_dwellings.attrs = {'name': 'Dwellings'}
        self.dwellings = df_dwellings

    # Gets dataframes with district information:
    def get_dataframes_with_district(self):
        return [df
                for df in self.get_all_dataframes()
                if 'District' in df.columns]

    # Gets all the dataframes and returns them in a list.
    def get_all_dataframes(self):
        return list(get_obj_attributes(self).values())

    # Gets the names of all columns in all datasets.
    def get_all_column_names(self):
        # Initialize the list of all columns from all the dataframes:
        all_columns = list()

        # Get all columns.
        for df in self.get_all_dataframes():
            all_columns += df.columns.to_list()

        return list(sorted(set(all_columns)))

    # Displays the dataframes with specified names.
    def display_dataframes(self, names=None):
        for attribute_name, df in get_obj_attributes(self).items():
            if (names is None) or (df.attrs["name"] in names):
                printmd(f'\n**{df.attrs["name"]}**:')
                display(df)

    # Gets the names of the dataframes in the object.
    def get_dataframe_names(self):
        return [df.attrs['name'] for df in self.get_dataframes()]

    # Displays the names of the dataframes in the object.
    def display_dataframe_names(self):
        # Display all the dataframes.
        df_names = self.get_dataframe_names()
        printmd(f'The dataframes in the object are:')
        printmd(f'***{"***, ***".join(df_names)}***')

### Plotting functions

In [6]:
# A function that applies default formatting to an axes.
def format_axes(axes: plt.Axes,
                keep_box=False):
    if not keep_box:
        axes.spines['top'].set_color('white')
        axes.spines['right'].set_color('white')

    axes.set_facecolor("white")


# A function that applies default formatting to annotation
# of an axes.
def format_axes_annotation(axes: plt.Axes):
    axes.xaxis.label.set_fontsize(14)
    axes.yaxis.label.set_fontsize(14)
    axes.title.set_fontsize(16)


# A function for creating common x-label for the figure.
def figure_x_label(figure: plt.Figure,
                   label: str,
                   y_position=0.04,
                   font_size=16):
    figure.text(0.5, y_position, label,
                ha='center',
                fontdict={'size': font_size})


# A function for creating common y-label for the figure.
def figure_y_label(figure: plt.Figure,
                   label: str,
                   x_position=0.04,
                   font_size=16):
    figure.text(x_position, 0.5, label,
                va='center',
                rotation='vertical',
                fontdict={'size': font_size})


# A function that draws a horizontal line across the entire axes.
def draw_threshold(value: float,
                   axes: plt.Axes,
                   linewidth=1,
                   linestyle='-',
                   color=None,
                   title=None):
    
    # Get axes limits and ranges.
    x_min, x_max = axes.get_xlim()
    x_range = x_max - x_min
    y_min, y_max = axes.get_ylim()
    y_range = y_max - y_min
    
    # Plot the threshold line.
    axes.plot([x_min, x_max], [value, value],
              linewidth=1,
              linestyle='-',
              color=color)
    
    # Write a title above the threshold line
    if title is not None:
        axes.text(x_min + 0.01 * x_range,
                  value + 0.02 * y_range,
                  title)

###  Dataframe functions

In [7]:
# A function that gets column names of a dataframe.
def get_df_columns(df, exclude=None):
    # If columns to be excluded have not been defined,
    # represent it as an empty list.
    if exclude is None:
        exclude = list()

    # If the columns to be excluded are not specified using a list
    # or a tuple, represent them as a list.
    elif not isinstance(exclude, (list, tuple)):
        exclude = [exclude]

    # Return all column names except the ones to exclude.
    return [column for column in df.columns.to_list()
            if column not in exclude]

# A function that resets the names of indices
def reset_df_index_names(df):
    return (
        df
        .rename_axis(['' for level in range(df.columns.nlevels)],
                     axis="columns")
        .rename_axis(['' for level in range(df.index.nlevels)],
                     axis="rows")
    )


# A function for balancing a dataframe so that the number of rows
# containing each value present in the designated column will be the same.
def balance_dataframe(df: pd.DataFrame, column_name):
    # Get the number of crimes for the least frequent crime.
    lowest_frequency = df['Category'].value_counts().min()

    # Create an empty dataframe for storing the balanced data
    df_balanced = pd.DataFrame()

    # For each value in column, randomly choose the number of samples
    # that corresponds to the least frequent value in the column.
    for value in df[column_name].unique():
        df_balanced = df_balanced.append(
            df
            .loc[df[column_name] == value]
            .sample(lowest_frequency)
        )

    return df_balanced


# A function that evaluates a dictionary of models on data from
# a pandas dataframe.
def evaluate_models(models: dict,
                    df: pd.DataFrame,
                    predictor_labels: list,
                    target_label: str,
                    test_size=0.33):

    # Get the dataset.
    X = df.loc[:, predictor_labels].values
    y = df.loc[:, target_label].values

    # Split the dataset into a test and training set.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=32)

    # Fit the models to the data.
    for model_name, model in models.items():

        # Print the name of the model.
        printmd(f'*__{model_name}:__*')

        # Train the model on the training set.
        model.fit(X_train, y_train)

        # Get the predictions on the test set.
        predictions = model.predict(X_test)

        # Print the classification report.
        print(classification_report(y_test, predictions,
                                    zero_division=0,
                                    digits=4))


# A function that sorts the columns in alphabethical order
# and puts the user-chosen columns first
def df_sort_columns(df: pd.DataFrame,
                    first_columns=['Year',
                                   'Quarter',
                                   'District',
                                   'District type',
                                   'Sex',
                                   'Age']):

    # Define a function that moves the chosen element to the
    # front of the list.
    def move_to_front(element, list_):
        if element in list_:
            list_.insert(0, list_.pop(list_.index(element)))

    # Make sure that the columns that are to be put in front
    # are represented as a list.
    if first_columns is None:
        first_columns = list()

    # Sort the columns in alphabetical order.
    sorted_columns = list(df.columns)
    sorted_columns.sort()

    # Move the user-chosen columns to the front.
    for column in first_columns[::-1]:
        move_to_front(column, sorted_columns)

    # Assign the ordered columns to the dataframe.
    df = df[sorted_columns]

    return df


# A function that creates a new column representing data in 'value_columns'
# for every unique value in 'category_columns'.
def df_create_column_for_each_unique_value(df,
                                           category_columns,
                                           value_columns,
                                           aggfunc='first'):

    # Always represent category and value columns as a list or tuple.
    if not isinstance(category_columns, (list, tuple)):
        category_columns = [category_columns]

    if not isinstance(value_columns, (list, tuple)):
        value_columns = [value_columns]

    # Create a colum order for grouping so that all the value columns
    # come last and category columns second last. We leave out 1
    # value column for the result
    cat_and_value_columns = category_columns + value_columns
    column_order = (
        get_df_columns(df, exclude=cat_and_value_columns)
        + cat_and_value_columns[:-1]
    )

    # Create columns from unique values by grouping and unstacking.
    df = (
        df
        .groupby(column_order)
        .first()
        .unstack(list(np.arange(-len(cat_and_value_columns) + 1,
                                0)))
        .reset_index()
    )

    # Delete the names of the index levels
    df = df.rename_axis(['' for level in range(df.columns.nlevels)],
                        axis="columns")
    return df

# OLD IMPLEMENTATION
#     return (
#         df
#         .pivot_table(values=value_columns,
#                      index=get_df_columns(df,
#                                           exclude=category_columns + value_columns),
#                      columns=category_columns,
#                      aggfunc='first')
#         .reset_index()
#         .rename_axis(('', ''), axis="columns")
#     )

###  Data cleaning functions

In [8]:
# Define a function that loads a dataframe from
# kk.statistikbank.dk
def load_cph_df(path_csv):
    return pd.read_csv(path_csv,
                       sep='\t',
                       skiprows=0,
                       encoding='windows-1252')


# A function that loads a dataframe from multiple files.
def load_split_dataframe(paths_csv):

    # Load the first two dataframes.
    df_1 = load_cph_df(paths_csv[0])
    df_2 = load_cph_df(paths_csv[1])

    # Extract common columns.
    common_columns = list(set(df_1.columns)
                          & set(df_2.columns))

    # Load the whole dataframe.
    for idx, path_csv in enumerate(paths_csv):
        # Merge the dataframes.
        if idx == 0:
            df = load_cph_df(path_csv)
        else:
            df = df.merge(load_cph_df(path_csv),
                          left_on=common_columns,
                          right_on=common_columns)

    # Return the merged dataframe.
    return df


# A function that cleans the data about copenhagen district
def clean_districts(df: pd.DataFrame):

    # Continue only if the dataframe contains the column: "district"
    if 'district' not in df.columns:
        return df

    df = df.copy()

    # Define the dame of the column containing info about districts.
    district_column_name = 'district'

    # Extract the district names.
    districts = df[district_column_name]

    # Initialize the list for storing district types.
    district_types = list()

    # Define valid districts.
    valid_districts = ['Indre By', 'Østerbro', 'Nørrebro', 'Vesterbro/Kongens Enghave', 'Valby',
                       'Vanløse', 'Brønshøj-Husum', 'Bispebjerg', 'Amager Øst', 'Amager Vest']

    # Define valid districts.
    valid_polling_areas = ['1. Østerbro', '1. Øst', '1. Nord', '1. Syd', '1. Vest', '1. Nordvest',
                           '2. Sundbyvester', '2. Nord', '2. Syd', '2. Øst', '2. Vest', '3. Indre By',
                           '3. Nord', '3. Syd', '3. Øst',
                           '4. Sundbyøster', '4. Nord', '4. Syd', '4. Øst',
                           '5. Nørrebro', '5. Nord', '5. Nørrebrohallen',
                           '5. Syd', '5. Øst', '5. Vest', '5. Nordvest',
                           '6. Bispebjerg', '6. Vest', '6. Nord', '6. Øst', '6. Syd',
                           '7. Brønshøj', '7. Nord', '7. Syd', '7. Øst', '7. Vest',
                           '7. Nordvest', '7. Katrinedal', '7. Kirkebjerg', '7. Vanløse',
                           '8. Valby', '8. Nord', '8. Syd', '8. Vest', '8. Sydøst', '8. Midt',
                           '9. Vesterbro', '9. Nord', '9. Syd',
                           '9. Øst', '9. Vest', '9. Midt', '9. Sydhavn']

    # Assign type to each district.
    for district in districts:
        if (('District' in district) | (district in valid_districts)):
            district_types.append('District')

        elif (('Polling area' in district) | (district in valid_polling_areas)):
            district_types.append('Polling area')

        elif 'Copenhagen total' in district:
            district_types.append('Entire Copenhagen')

        else:
            district_types.append('Unknown')

    # Insert the column "district type" next to the district column.
    idx_district_column = df.columns.to_list().index(district_column_name)
    df.insert(idx_district_column + 1, 'district type', district_types)

    # Clean the names in the district - column.
    for token in ['District -', 'Polling area -']:
        df[district_column_name] = (
            df[district_column_name]
            .str.replace(token, '')
            .str.strip()
        )

    return df


# A function that cleans all the information about the time in the
# dataframes for Copenhagen.
def clean_years(df: pd.DataFrame,
                non_year_columns=None,
                value_name='Value'):

    if non_year_columns is not None:
        # If "non_year_columns" is an integer, regard the first n
        # columns represented by the integer as non-year columns
        if isinstance(non_year_columns, int):
            non_year_columns = df.columns[:non_year_columns]
    else:
        non_year_columns = [column for column in df.columns.to_list()
                            if column[0] not in ['1', '2']]

    # Create a row for each Year and Quarter.
    df = df.melt(
        id_vars=non_year_columns,
        var_name="Time",
        value_name=value_name
    )

    # If the time is represented by a year and a quarter,
    # create columns "Year" and "Quarter" from the column "Time".
    if 'Q' in df.loc[0, 'Time']:
        df[['Year', 'Quarter']] = (
            df
            .pop('Time').str.split('Q', 2, expand=True)
            .astype(int)
        )

    # If year is presented as "XXXX:YYYY", take the second value.
    elif ':' in df.loc[0, 'Time']:
        df['Year'] = (
            df
            .pop('Time')
            .str.split(':').str[-1]
            .astype(int)
        )

    # Otherwise, transform the year into an integer
    else:
        df['Year'] = df.pop('Time').astype(int)

    return df


# A fuction that removes all other quarters of the year than last.
def choose_latest_quarter(df: pd.DataFrame, value_name):
    # Run only if the dataframe has a column "Age"

    if 'Quarter' not in df.columns:
        return df

    return (
        df
        .sort_values(by=['Year', 'Quarter'])
        .groupby([column for column in df.columns
                  if column not in ['Quarter', value_name]],
                 as_index=False)
        .last()
    )


# A function that perfoms standard cleaning of a dataframe
# from kk.statistikbank.dk
def clean_cph_dataframe(df, value_name='Value', df_name=''):
    # Clean district information.
    df = clean_districts(df)

    # Clean time information.
    df = clean_years(df, value_name=value_name)

    # Capitalize column names.
    df.columns = [column.capitalize() for column in df.columns]

    # Standardize age intervals
    df_standardize_age(df, value_name)

    # Choose only the last available quarter of the year.
    df = choose_latest_quarter(df, value_name)  
    
    # Order the columns.
    df = df_sort_columns(df)
    
    # Name the dataframe.
    df.name = df_name

    return df


# A function that displays all basic stats about the dataframe.
def show_stats(df):
    # Show data types.
    display(df.dtypes.to_frame('Data types'))

    # Show missing values.
    display(
        df.isna()
        .sum()
        .to_frame('Number of missing values')
    )

    # Describe values in columns.
    display(df.describe(include='all'))


# A function that unifies age intervals in the databases from
# from Københavns Kommune:
def df_standardize_age(df: pd.DataFrame, sum_column):

    # Run only if the dataframe has a column "Age"
    if 'Age' not in df.columns:
        return df

    # Use 10-year intervals for age: Create the mapping.
    mapping_5_to_10_years = dict()
    for interval_min in range(0, 90, 10):
        map_from_low = f'{interval_min}-{interval_min + 4} years'
        map_from_high = f'{interval_min + 5}-{interval_min + 9} years'
        map_to = f'{interval_min}-{interval_min + 9}'

        mapping_5_to_10_years[map_from_low] = map_to
        mapping_5_to_10_years[map_from_high] = map_to

    # Define the mapping for all the possible versions of the 90+ age.
    mapping_5_to_10_years['90-99 + years'] = '90+'
    mapping_5_to_10_years['95-99 years'] = '90+'
    mapping_5_to_10_years['90-94 years'] = '90+'
    mapping_5_to_10_years['100 years and over'] = '90+'
    mapping_5_to_10_years['95+years'] = '90+'

    # Apply the mapping to the column "Age".
    df['Age'] = df['Age'].map(mapping_5_to_10_years)

    # Sum the values in the new bins. First, get the names
    # of all the columns that are NOT the column which will summed up.
    non_sum_columns = [column for column in df.columns
                       if column != sum_column]

    # Sum the values together in the age bins.
    df[sum_column] = (
        df
        .groupby(non_sum_columns)
        .transform('sum')
    )

    # Drop duplicate rows
    df.drop_duplicates(ignore_index=True, inplace=True)

    return df

<hr style="border:2px solid black"> </hr>

# Load data

In [16]:
%run ./website_main.ipynb
hello_wojciech()

4
hello, wojciech!


'hej!'

SyntaxError: invalid syntax (<ipython-input-15-0b648a5839ef>, line 1)